In [4]:
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################

#Need this so that the other files can get the imports.
#Put it in the same directory as the file you want to run.

######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################
######################################################################################################################################################################################################################################


import sys, traceback
print(sys.executable)
print(sys.version)
print(sys.version_info)
from multiprocessing import Pool, cpu_count
from concurrent.futures import ProcessPoolExecutor

import math
import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from spectral_cube import LazyMask
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              

import astrodendro #Change numpy.int to int, modified asscalar to just take an element
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
import gc
from astropy.convolution import Gaussian1DKernel
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astrodendro.analysis import PPVStatistic
import os
print(spectral_cube.__file__)

print(astrodendro.__file__)

from matplotlib.patches import Ellipse
import copy
from tqdm import tqdm
import radio_beam
from astropy.table import Table
print(np.__version__,"Numpy")
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
#%matplotlib widget
Num_per_kg= 6.0221409*10**23/(2.8*10**-3)#6.0221409*10**23/29.0180*10**-3#num/kg for h2

#Create a function that uses the dendrogram input to calculate all the quantities, and has the size and linewidth requirements of the Shetty paper
#Requires the computed dendrogram, the data from the line image, the velocity axis, and the data from the Continuum image, as well as metadata for the structures
#Finds Size, Linewidth, Luminosity, and Column Density of a structure for each structure and returns them in [[][]] arrays [Leaves][Branches]
#Continuum is in Jansky/Beam, Line data should have the unit specified in the metadata as 'data_unit'
gal="GC"

dist_cmz = 8.178*10**-3*u.Mpc

def Dendro_Arrays(Dendrogram,LineData,DataVel,ContData,metadata,ColD = True,beam_size=999,beam_req = 999999,Trunks=True,max_size=18,edge_cases=False):
    SizeA,SigmaA,LuminA,CDA,SIDS,MOM0_FLUX,Distances,V_rms_err = [[],[],[],[]],[[],[],[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
    print(metadata)
    
    d_copy= Dendrogram
    #catalog = astrodendro.ppv_catalog(d, metadata)
    if (gal =="GC"):
        center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
        dist_val=8.178*10**3
    else:
        center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')
        dist_val=3.5*10**6
    
    center_ra_pix,center_dec_pix = int(metadata['wcsu'][:][:][0].world_to_pixel(center)[0]),int(metadata['wcsu'][:][:][0].world_to_pixel(center)[1])
    
    sliced= LineData[6]
    CubeShape = np.shape(sliced)
    DataShape=[[0,0],[0,0]]#The part of the cube that actually has data

    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[lmi])
        if(allData>0 or allData<0):
            DataShape[0][0] = lmi+3
            break
    for lmi in range(CubeShape[0]):
        allData=np.nansum(sliced[CubeShape[0] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[0][1] = CubeShape[0] - lmi -3
            break
    for lmi in range(CubeShape[1]):
        allData=(sliced[DataShape[0][0],lmi])
        if(allData>0 or allData<0):
            DataShape[1][0] = lmi+3
            break
    for lmi in range(CubeShape[1]):
        allData=(sliced[DataShape[0][0],CubeShape[1] - lmi -1])
        if(allData>0 or allData<0):
            DataShape[1][1] = CubeShape[1] - lmi -3
            break
    for t in Dendrogram.all_structures: 

        I = t.indices()
        Cont = True
        if t.is_branch:
                if t.parent==None:
                    
                    if(Trunks):
                        Cont = True
                    else:
                        Cont = False
                else:
                    Cont=True
                    
        for lmi in range(len(I[0])):
            if(I[1][lmi]<=DataShape[0][0] or I[1][lmi]>=DataShape[0][1] or I[2][lmi]<=DataShape[1][0] or I[2][lmi]>=DataShape[1][1]):
                #print(I[1][lmi],I[0][lmi])
                if edge_cases:
                    Cont=True
                else:
                    Cont=False
                break
                

        if(Cont):
            s = PPVStatistic(t,metadata=metadata)
            s_radius = s.radius
            s_v_rms = s.v_rms
            
            
            if((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*dist_val/10**6/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01):
            
            

                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                
                #The flux from the continuum
                #Convert to Jansky from Jansky per beam:
                if(ColD ==True):
                    Cont_Flux=0

                    proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                    for lmi in range(len(proj)):

                        Cont_Flux+=ContData[proj[lmi]]
                    Cont_Flux=Cont_Flux/(metadata['beam_area_ratioc']*(2*np.sqrt(2*np.log(2))))*u.pix**2*u.beam/u.beam*u.Jy#SHould be input as Jansky /beam and will be converted to Jansky, then to unitless. The beam is changed from FWHM to Gaussian
                    Dust_Column = Flux_to_Mass(Cont_Flux)*Num_per_kg/((s_radius*np.pi/180*dist_cmz.value/u.deg)**2*(3.086*10**24)**2)/np.pi*(1.989*10**30*u.kg/u.M_sun)/u.kg
                    
                else:
                    Dust_Column=0
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"])#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                
                
                
                Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2)*metadata['spatial_scale']*np.pi/180*dist_cmz.value*10**6/u.deg#pc dist from barycenter
                
                
                V_err= Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m,NF=1,iterations=5,metadata=metadata)
                
                
                if(t.is_leaf):

                    SizeA[0].append((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))) #define size as astrodendro
                    SigmaA[0].append((float(s_v_rms/u.km*u.s)))#
                    CDA[0].append(float(Dust_Column))
                    LuminA[0].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[0].append(float(t.idx))
                    MOM0_FLUX[0].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[0].append(float(Distance))
                    V_rms_err[0].append(float(V_err))
                if(t.is_branch	):

                    SizeA[1].append((float((s_radius*np.pi/180*dist_val/10**6/u.deg)))) #define size as astrodendro
                    SigmaA[1].append((float(s_v_rms/u.km*u.s)))#
                    CDA[1].append(float(Dust_Column))
                    LuminA[1].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[1].append(float(t.idx))
                    MOM0_FLUX[1].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[1].append(float(Distance))
                    V_rms_err[1].append(float(V_err))
                del s
                    
                    
                    
    SizeA[0] = np.array(SizeA[0],dtype=type(1.))
    SizeA[1] = np.array(SizeA[1],dtype=type(1.))
    SizeA[2] = np.array(SizeA[2],dtype=type(1.))
    SizeA[3] = np.array(SizeA[3],dtype=type(1.))
    SigmaA[0] = np.array(SigmaA[0],dtype=type(1.))
    SigmaA[1] = np.array(SigmaA[1],dtype=type(1.))
    SigmaA[2] = np.array(SigmaA[2],dtype=type(1.))
    SigmaA[3] = np.array(SigmaA[3],dtype=type(1.))
    CDA[0] = np.array(CDA[0],dtype=type(1.))
    CDA[1] = np.array(CDA[1],dtype=type(1.))
    LuminA[0] = np.array(LuminA[0],dtype=type(1.))
    LuminA[1] = np.array(LuminA[1],dtype=type(1.))
    SIDS[0] = np.array(SIDS[0],dtype=type(1.))
    SIDS[1] = np.array(SIDS[1],dtype=type(1.))
    MOM0_FLUX[0] = np.array(MOM0_FLUX[0],dtype=type(1.))
    MOM0_FLUX[1] = np.array(MOM0_FLUX[1],dtype=type(1.))
    Distances[0] = np.array(Distances[0],dtype=type(1.))
    Distances[1] = np.array(Distances[1],dtype=type(1.))
    V_rms_err[0] = np.array(V_rms_err[0],dtype=type(1.))
    V_rms_err[1] = np.array(V_rms_err[1],dtype=type(1.))
    
    return np.array(SizeA),np.array(SigmaA),np.array(CDA),np.array(LuminA),np.array(SIDS),np.array(MOM0_FLUX),np.array(Distances),np.array(V_rms_err)

#Make a function to make an image 

#Data to plot, minimum of color bar, maximum, WCS projection for coords, and position of the image in the larger figure
def Make_Plot(Name,Name2,Data,vmin,vmax,WCS,rows,columns,index,show):
    

    ax = pylab.subplot(rows,columns,index,projection=WCS) 
    RA = ax.coords[0]                                                                  # 
    Dec = ax.coords[1]
    im = pylab.imshow(Data,vmin=vmin,vmax=vmax,cmap='rainbow')
    RA.set_ticks(size=-3)                                                                                      
    Dec.set_ticks(size=-3) 
    RA.set_ticklabel(exclude_overlapping=True) 
    Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
    
    if(gal=="NGC253"):
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
    else:
        pylab.xlabel('Glon',fontsize=20,labelpad=1)                               
        pylab.ylabel('Glat',fontsize=20,labelpad=1)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 15)    
    cb=pylab.colorbar(im,fraction=0.1,pad=0.0)                                     
    cb.set_label(label=Name,fontsize=10,rotation=270,labelpad=20) 
    cb.ax.tick_params(which = 'major', labelsize = 10)   
    pylab.annotate(text=Name2,fontsize=10,xy=(0.02,1.05),xycoords="axes fraction")  
    if(show==True):
        pylab.show()
def Make_Plot_Anno(Name,Name2,Data,vmin,vmax,WCS,rows,columns,index,show,pos1,pos2):
    

    ax = pylab.subplot(rows,columns,index,projection=WCS) 
    RA = ax.coords[0]                                                                  # 
    Dec = ax.coords[1]
    im = pylab.imshow(Data,vmin=vmin,vmax=vmax,cmap='rainbow')
    RA.set_ticks(size=-3)                                                                                      
    Dec.set_ticks(size=-3) 
    RA.set_ticklabel(exclude_overlapping=True) 
    Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
    if(gal=="NGC253"):
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
    else:
        pylab.xlabel('Glon',fontsize=20,labelpad=1)                               
        pylab.ylabel('Glat',fontsize=20,labelpad=1)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 15)    
    cb=pylab.colorbar(im,fraction=.1,pad=0.0)                                     
    cb.set_label(label=Name,fontsize=10,rotation=270,labelpad=20) 
    cb.ax.tick_params(which = 'major', labelsize = 10)   
    pylab.annotate(text=Name2,fontsize=10,xy=(pos1,pos2),xycoords="axes fraction")  
    if(show==True):
        pylab.show()
        
        
#Put this up here for the column density map
def Flux_to_Mass(flux):
    J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy
    flux_erg = flux*J_to_e
    if(gal=="NGC253"):
        L = 4*np.pi*(3.5*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    else:
        L = 4*np.pi*(8.178*10**-3*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    
    
    a_850 = 6.7*10**19*u.erg/u.s/u.Hz/u.M_sun #6.7+-1.7
    
    M_mol = L/a_850#Just in Solar mass*1.989*10**30*u.kg/u.M_sun #Determines mass of the cont for 850 in kg
    return M_mol
def Flux_to_Lum(flux):
    J_to_e = 10**-23*u.erg/u.s/u.cm**2/u.Hz/u.Jy
    flux_erg = flux*J_to_e
    if(gal=="NGC253"):
        L = 4*np.pi*(3.5*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    else:
        L = 4*np.pi*(8.178*10**-3*3.086*10**24)**2*flux_erg*u.cm**2 #Megaparsec is converted to cm
    
    return L


def Get_V_rms_err(dend1,struct,idx,m,NF,iterations,metadata):
    
    
    vs=[]
    np.random.seed((99)**2*123)
    for llll in range(iterations):
        
        #print(llll)
        s = dend1.__getitem__(idx)
        #s = struct#copy.deepcopy(struct)
        #s2 = struct#copy.deepcopy(struct)
        npixels = np.product(np.shape(s.values()))
        #print(np.shape(s.values()),s.values())
        
        additional_noise = np.random.normal(0., m*NF, npixels)
        additional_noise = np.reshape(additional_noise, np.shape(s.values()))
        #add or subract noise to the values and calculate the v rms, them find the std of that array and
        # call that the uncertainty in v rms for a structure
        dat1P = dend1.data[s.indices()]
        dend1.data[s.indices()]+= additional_noise
        s = dend1.__getitem__(idx)
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms/u.km*u.s))
        dend1.data[s.indices()]= dat1P#reset the dend data
        
        dend1.data[s.indices()]-= additional_noise
        #s._values+=additional_noise
        #print(s.values(),s._values)
        
        #s2._values-=additional_noise
        s = dend1.__getitem__(idx)
        #print(dat1P[0],s._values[0],"kaasl")
        vs.append(float(PPVStatistic(s,metadata=metadata).v_rms/u.km*u.s))
        
        del s
        #del s2
        
    v_rms_std = np.nanstd(vs)
    #print(v_rms_std)
    return v_rms_std

#Return a cropped cube for some ra and dec, also crops the velocity axis if needed (0 for no crop)
def Crop(cube,WCS,Np1,Np2,BadVel,D2):
    NraDP1 = [int(WCS.world_to_pixel(Np1)[0]),int(WCS.world_to_pixel(Np1)[1])]
    NraDP2 = [int(WCS.world_to_pixel(Np2)[0]),int(WCS.world_to_pixel(Np2)[1])]
    if(D2==False):
        return cube[BadVel:np.shape(cube)[0]-BadVel,NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]
    if(D2==True):
        return cube[NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]

    
def Read_Clusters(FileName):
    
    sh= len(np.genfromtxt(FileName,usecols=0))
    Data=[]
    for lmi in range(50):
        try:
            Data.append(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m")))
            #print(np.genfromtxt(FileName,usecols=lmi,dtype=type("2d4m"),skip_header=1))
        except:
            pass
    return Data
def Find_Clusters_NGC(Data):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
            
    return IDs,RAs,Decs,R_deconv
#Take the cont in Jy and find the HWHM from the structures in the catalog
def Find_Clusters(Data,wcs,Cont_Data,header):
    for lmi in range(len(Data)):
        if "ID" in Data[lmi]:
            IDs= Data[lmi][1:9999]
        if "RA" in Data[lmi]: 
            RAs= Data[lmi][1:9999]
        if "Dec" in Data[lmi]:
            Decs= Data[lmi][1:9999]
        if "r_deconv" in Data[lmi]: 
            R_deconv= Data[lmi][1:9999]#pc
        if "glon" in Data[lmi]: 
            glons= Data[lmi][1:9999]#
        if "glat" in Data[lmi]: 
            glats= Data[lmi][1:9999]#
        if "herschel_column" in Data[lmi]: 
            CD= (Data[lmi][1:9999])#pc
            
        if "flux_integrated" in Data[lmi]: 
            Flux_1p3mm= Data[lmi][1:9999]#pc
    #remove nan 
    for lmii in range(len(CD)):
        try:
            if CD[lmii]=='np.nan':
                CD= np.delete(CD, lmii)
                Flux_1p3mm= np.delete(Flux_1p3mm, lmii)
                IDs= np.delete(IDs, lmii)
                glats= np.delete(glats, lmii)
                glons= np.delete(glons, lmii)
                
        except:
            CD = np.array(CD,dtype=type(1.2**5))#float
            break
    glats_New=[]
    glons_New=[]
    CDs_New=[]
    IDs_New=[]
    Flux_1p3mm_New=[]

    #print(CD,sorted(CD),type(CD),type(CD[0]))
    nth = sorted(CD)[len(CD)-34]#34 most dense leaves
    #print(nth,"A",CD,sorted(CD))
    for lmj in range(len(CD)):
        if CD[lmj]>nth:
            glats_New.append(glats[lmj])
            glons_New.append(glons[lmj])
            CDs_New.append(CD[lmj])
            IDs_New.append(int(IDs[lmj]))
            Flux_1p3mm_New.append(Flux_1p3mm[lmj])
    HWHM_rad = []      
    #print(Flux_1p3mm_New,glats_New,glons_New,CDs_New,IDs_New)
    for lmi in range(len(CDs_New)):
        glat = glats_New[lmi]
        glon = glons_New[lmi]
        Flux = float(Flux_1p3mm_New[lmi])#INtegerated flux in jy
        
        Circle_R = 0
        distance = 8.178*10**-3*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(l=float(glon)*u.deg, b=float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        
        while(True):
            Circle_R += .01
            #pixels=[(p1,p2)]
            pixels=[(p2,p1)]#Goes lat then long for the cont data
            #print(p1,p2)
            #print(np.shape(Cont_Data[p2-50:p2+50]))
            #print(np.shape(Cont_Data[50,p1-50:p1+50]))
            for lmii in range(np.shape(Cont_Data[p2-50:p2+50])[0]):
                for lmjj in range(np.shape(Cont_Data[p2-50+lmii,p1-50:p1+50])[0]):
                    #Find pixels within the circle around the center (excude the center since its there already)
                    #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                    if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                        pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
                        
            
            
            sum_flux=0
            for lmkk in range(len(pixels)):
                sum_flux += (Cont_Data[pixels[lmkk]])
            #print(p1,p2,glat,glon,np.shape(Cont_Data),pixels,Cont_Data[pixels[0]],Flux,sum_flux,Circle_R)
            if sum_flux>Flux/2:
                HWHM_rad.append(Circle_R)#Pc
                break
                
    return HWHM_rad,CDs_New,glons_New,glats_New,IDs_New

#Return masked data around clusters or one pc around clusters
def Mask_Clusters_NGC(HWHM,wcs,header,unmasked_data,ras,decs,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        ra = ras[lmi]
        dec = decs[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = 3.5*u.Mpc
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(str(ra),str(dec), frame='icrs')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,1,True)
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,2,True)


def Mask_Clusters_CMZ(HWHM,wcs,header,unmasked_data,glons,glats,One_Pc=False,One_Pc_Size=1,HWHM_Fac=1):
    
    Masked_Data=copy.deepcopy(unmasked_data)
    for lmi in range(len(HWHM)):
        glon = glons[lmi]
        glat = glats[lmi]
                
        Circle_R = HWHM[lmi]*HWHM_Fac
        if(One_Pc):
            
            Circle_R=One_Pc_Size
        distance = dist_cmz
        
        pixel_res = abs(header['cdelt1'])*np.pi/180*distance*10**6/u.Mpc*u.pc # cdelt in deg, goes to res in pc
        
        #sky = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
        sky = SkyCoord(float(glon)*u.deg,float(glat)*u.deg, frame='galactic')
        #center = SkyCoord(l=359.94487501*u.degree,b=-00.04391769*u.degree, frame='galactic')
        p1,p2 = int(wcs.world_to_pixel(sky)[0]),int(wcs.world_to_pixel(sky)[1]) #Ra,dec
        


        #pixels=[(p1,p2)]
        pixels=[(p2,p1)]#Goes lat then long for the cont data
        #print(p1,p2)
        #print(np.shape(Cont_Data[p2-50:p2+50]))
        #print(np.shape(Cont_Data[50,p1-50:p1+50]))
        for lmii in range(np.shape(unmasked_data[0,p2-50:p2+50])[0]):
            for lmjj in range(np.shape(unmasked_data[0,p2-50+lmii,p1-50:p1+50])[0]):
                #Find pixels within the circle around the center (excude the center since its there already)
                #print(np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res,lmjj)
                
                if np.sqrt((lmii-50)**2+(lmjj-50)**2)*pixel_res.value < Circle_R and lmjj!=50:
                    pixels.append((lmjj-50+p2,lmii-50+p1))#Goes lat then long
        
        for lmi in range(len(unmasked_data)):
            
            for lmj in range(len(pixels)):
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
                Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]]=np.nan
                #print(Masked_Data[lmi,pixels[lmj][0],pixels[lmj][1]],lmi,pixels,np.shape(Masked_Data))
     
    return Masked_Data
            
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,1,True)
#Make_Plot("Tes","Test2",Q.moment0().hdu.data,0,0,Q.wcs[:][:][0],2,2,2,True)

def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


from astropy.utils import NumpyRNGContext
def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field
restfreq = 345.79598990 * u.GHz







#For the ngc253 data we need another function

def Dendro_Arrays_NGC(Dendrogram,LineData,DataVel,ContData,metadata,ColD = True,beam_size=999,beam_req = 999999,Edge_Cases=True,Trunks=True,max_size=1):
    SizeA,SigmaA,LuminA,CDA,SIDS,MOM0_FLUX,Distances,V_rms_err = [[],[],[],[]],[[],[],[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]],[[],[]]
    print(metadata)
    
    d_copy= Dendrogram
    #catalog = astrodendro.ppv_catalog(d, metadata)
    center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')
    center_ra_pix,center_dec_pix = int(metadata['wcsu'][:][:][0].world_to_pixel(center)[0]),int(metadata['wcsu'][:][:][0].world_to_pixel(center)[1])
    sliced= LineData[6]
    CubeShape = np.shape(sliced)
    for t in Dendrogram.all_structures: 

        I = t.indices()
        Cont = True
        if t.is_branch:
                if t.parent==None:
                    if Trunks:
                        Cont=True
                    else:
                        Cont=False
                else:
                    Cont = True
        
        for lmi in range(len(I[0])):
            NansNE=0
            NansSE=0
            NansNW=0
            NansSW=0
            Length = 10
            #I[1][lmi+10]>
            if Edge_Cases==False:
                for lmj in range(Length):
                    #Check four 45 degree prongs from each point and see if they have at least 7 nans in 10 pixels. If that happens its too close to the boundary
                    try:

                        if(sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]-lmj]<0 ):
                            pass
                        else:
                            NansSE+=1
                        if(sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]-lmj]<0 ):
                            pass
                        else:
                            NansSW+=1
                        if(sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]-lmj,I[2][lmi]+lmj]<0 ):
                            pass
                        else:
                            NansNW+=1
                        if(sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]>0 or sliced[I[1][lmi]+lmj,I[2][lmi]+lmj]<0 ):
                            pass
                        else:
                            NansNE+=1
                    except:
                        #only fails if the I goes close to the boundary of the cube and tries to get a pixel outside the cube
                        Cont = False
                if(NansNE>Length-3 or NansNW>Length-3 or NansSE>Length-3 or NansSW>Length-3):
                    Cont = False

                    break

        if(Cont):
            s = PPVStatistic(t,metadata=metadata)
            s_radius = s.radius
            s_v_rms = s.v_rms
            #print(np.nanmax(I[0]) - np.nanmin(I[0]) )
            #if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01 and (np.nanmax(I[0]) - np.nanmin(I[0]))*(float((s_radius*np.pi/180*3.5/u.deg)))*10**6 >3*3):
            if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01 and (np.nanmax(I[0]) - np.nanmin(I[0]))>3):
            #if((float((s_radius*np.pi/180*3.5/u.deg)))*10**6<max_size and (float((s_radius*np.pi/180*3.5/u.deg)))*10**6>beam_size*beam_req and (float(s_v_rms/u.km*u.s))>.01):
            
            

                nproj_pix=len(set(zip(*tuple(I[i] for i in [1,2]))))
                v_IWM = np.nansum(LineData[I]*(DataVel[I[0]])/u.km*u.s)/np.nansum(LineData[I])
                sig_Sh = np.sqrt(np.nansum(LineData[I]*((DataVel[I[0]])/u.km*u.s-v_IWM)**2)/np.nansum(LineData[I])) 
                
                #The flux from the continuum
                #Convert to Jansky from Jansky per beam:
                if(ColD ==True):
                    Cont_Flux=0

                    proj = tuple(set(zip(*tuple(I[i] for i in [1,2]))))
                    for lmi in range(len(proj)):

                        Cont_Flux+=ContData[proj[lmi]]
                    Cont_Flux=Cont_Flux/(metadata['beam_area_ratioc']*(2*np.sqrt(2*np.log(2))))*u.pix**2*u.beam/u.beam*u.Jy#SHould be input as Jansky /beam and will be converted to Jansky, then to unitless. The beam is changed from FWHM to Gaussian
                    Dust_Column = Flux_to_Mass(Cont_Flux)*Num_per_kg/((s_radius*np.pi/180*3.5/u.deg)**2*(3.086*10**24)**2)/np.pi*(1.989*10**30*u.kg/u.M_sun)/u.kg
                else:
                    Dust_Column=0
                if(str(Dust_Column) == str(np.nan) or str(Dust_Column)==str(np.inf)):
                    Dust_Column=0
                lum = Flux_to_Lum(s.flux)
                s_flux = s.flux

                Index = tuple(I[i] for i in [0,1,2])
                K_Km_s_Flux=np.nansum(LineData[Index]*metadata["velocity_scale"])#Find the total flux from the structures in K km/s, assuming the input data is in K as it should be, 
                
                
                
                
                Distance = np.sqrt((float(s.x_cen/u.pix)-center_ra_pix)**2+(float(s.y_cen/u.pix)- center_dec_pix)**2)*metadata['spatial_scale']*np.pi/180*3.5*10**6/u.deg#pc dist from barycenter
                
                
                V_err= 0#Get_V_rms_err(dend1=d_copy,idx=int(t.idx),struct=t,m=m,NF=1,iterations=5,metadata=metadata)
                
                
                if(t.is_leaf):

                    SizeA[0].append((float((s_radius*np.pi/180*3.5/u.deg)))) #define size as astrodendro
                    SigmaA[0].append((float(s_v_rms/u.km*u.s)))#
                    CDA[0].append(float(Dust_Column))
                    LuminA[0].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[0].append(float(t.idx))
                    MOM0_FLUX[0].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[0].append(float(Distance))
                    V_rms_err[0].append(float(V_err))
                if(t.is_branch	):

                    SizeA[1].append((float((s_radius*np.pi/180*3.5/u.deg)))) #define size as astrodendro
                    SigmaA[1].append((float(s_v_rms/u.km*u.s)))#
                    CDA[1].append(float(Dust_Column))
                    LuminA[1].append(float(lum*u.Hz*u.s/u.erg))
                    SIDS[1].append(float(t.idx))
                    MOM0_FLUX[1].append(float(K_Km_s_Flux*u.s/u.km))
                    Distances[1].append(float(Distance))
                    V_rms_err[1].append(float(V_err))
                del s
                    
                    
                    
    SizeA[0] = np.array(SizeA[0],dtype=type(1.))
    SizeA[1] = np.array(SizeA[1],dtype=type(1.))
    SizeA[2] = np.array(SizeA[2],dtype=type(1.))
    SizeA[3] = np.array(SizeA[3],dtype=type(1.))
    SigmaA[0] = np.array(SigmaA[0],dtype=type(1.))
    SigmaA[1] = np.array(SigmaA[1],dtype=type(1.))
    SigmaA[2] = np.array(SigmaA[2],dtype=type(1.))
    SigmaA[3] = np.array(SigmaA[3],dtype=type(1.))
    CDA[0] = np.array(CDA[0],dtype=type(1.))
    CDA[1] = np.array(CDA[1],dtype=type(1.))
    LuminA[0] = np.array(LuminA[0],dtype=type(1.))
    LuminA[1] = np.array(LuminA[1],dtype=type(1.))
    SIDS[0] = np.array(SIDS[0],dtype=type(1.))
    SIDS[1] = np.array(SIDS[1],dtype=type(1.))
    MOM0_FLUX[0] = np.array(MOM0_FLUX[0],dtype=type(1.))
    MOM0_FLUX[1] = np.array(MOM0_FLUX[1],dtype=type(1.))
    Distances[0] = np.array(Distances[0],dtype=type(1.))
    Distances[1] = np.array(Distances[1],dtype=type(1.))
    V_rms_err[0] = np.array(V_rms_err[0],dtype=type(1.))
    V_rms_err[1] = np.array(V_rms_err[1],dtype=type(1.))
    
    return np.array(SizeA),np.array(SigmaA),np.array(CDA),np.array(LuminA),np.array(SIDS),np.array(MOM0_FLUX),np.array(Distances),np.array(V_rms_err)




#repo func

def Reproject_To_Region(File,Prime_Beam,Gal,ovs,FOV,Force_Origins=False,Force_Value_x=0,Force_Value_y=0,i_step=30,Cube_Name_Save=''):
    
    
    #Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high)?
    

    
    scB = SpectralCube.read(File) #[:] 

    scB = scB.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    
    #Put in the right system or not
    scB.allow_huge_operations=True
    vel,RA,Dec = scB.world[:,0,0]

    

    for i in range(int(len(scB)/i_step) +1):
        try:

            sc = scB[i*i_step:i*i_step+i_step]

            print('start')

            HI = sc.header
            Nres=Prime_Beam
            if(gal=="NGC253"):
                dist=3.5*10**6
                sc = sc.spectral_slab(0. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about  
            if(gal=="GC"):
                dist=8.178*10**3#pc
                sc = sc.spectral_slab(-325. *u.km / u.s, 326. *u.km / u.s)  # Crop out velocities we don't care about  
            res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc

            beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)


            sc.allow_huge_operations=True







            #Now that we have a circular beam, this can be done easily:
            print('fov crop start')

            if(gal == 'GC'):

                cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
                cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
                center = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')

                center_ra_pix,center_dec_pix = int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])

                PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*8.178*10**3))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*8.178*10**3))/2]


                pixels = np.zeros(np.shape(sc))  
                print(PixFov)
                #
                #Find all pixels in the fov
                #
                for lmi in range(np.shape(sc)[0]):
                    for lmj in range(np.shape(sc)[1]):
                        for lmk in range(np.shape(sc)[2]):

                            up_pixels = abs(lmj-center_dec_pix)#Should not be over the fov in the upwards direction (relative to 0 degrees)
                            side_pixels = abs(lmk-center_ra_pix)#Should not be over the fov in the side-side direction (relative to 0 degrees)

                            if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):

                                pixels[lmi][lmj][lmk] = 1#good
            elif(gal == 'NGC253'):
                #
                #
                #
                center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')
                cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
                cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])

                center_ra_pix,center_dec_pix = [int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])]
                PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*3.5*10**6))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*3.5*10**6))/2]
                print(center_ra_pix,center_dec_pix)
                print(PixFov)
                pixels = np.zeros(np.shape(sc))
                print(np.shape(sc))

                for lmj in range(np.shape(sc)[1]):
                    for lmk in range(np.shape(sc)[2]):
                        hypo = np.sqrt(((lmj-center_dec_pix)**2) + (lmk-center_ra_pix)**2)

                        if (lmj-center_dec_pix!=0):
                            ang = np.arctan(abs(lmk-center_ra_pix)/abs(lmj-center_dec_pix))#Find angle to the center
                        else:
                            ang = np.pi/2
                        if(lmk>center_ra_pix and lmj>center_dec_pix):
                            ang*=-1
                        elif(lmk<center_ra_pix and lmj<center_dec_pix):
                            ang*=-1
                        elif(lmk>center_ra_pix and lmj<center_dec_pix):
                            if ang >57*np.pi/180:

                                ang= -33*np.pi/180+147*np.pi/180-ang#coming from the opposite end of the ra axis now, but projecting still to 33 deg from north.


                            else:

                                pass
                        up_pixels = abs(hypo*np.cos(abs((33*np.pi/180)+ang)))#Should not be over the fov in the upwards direction (relative to 0 degrees)
                        side_pixels = abs(hypo*np.sin(abs((33*np.pi/180)+ang)))#Should not be over the fov in the side-side direction (relative to 0 degrees)

                        if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                            for lmi in range(np.shape(sc)[0]):
                                pixels[lmi][lmj][lmk] = 1#good






            bp = np.where(pixels!=1)

            #Mask teh pixels outside the fov
            scCopy = sc.hdu

            scCopy.data[bp]=np.nan
            sc = SpectralCube.read(scCopy)
            del scCopy


            sc.allow_huge_operations=True
            datn = sc.hdu.data
            sx,sy,ex,ey=0,0,0,0
            for lmi in range(np.shape(datn[0,:,:])[0]):

                if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
                    print("F",lmi)
                    break
                for lmj in range(np.shape(datn[0,:,:])[1]):

                    if(sx==0):            
                        if(np.nanmean(datn[0,lmi,:])>0 or np.nanmean(datn[0,lmi,:])<0):
                            sx=lmi


                    if(sy==0):
                        if(np.nanmean(datn[0,:,lmj])>0 or np.nanmean(datn[0,:,lmj])<0):
                            sy=lmj

                    if(ex==0):
                        if(np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])<0):
                            ex=np.shape(datn[0,:,:])[0]-lmi-1

                    if(ey==0):
                        if(np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])>0 or np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])<0):
                            ey=np.shape(datn[0,:,:])[1]-lmj-1

                    if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                        break
            scP = sc[:,sx:ex,sy:ey]
            scP_Hdu=scP.hdu
            zeros=((scP_Hdu.data[:,:,:]==0))
            bp = np.where(zeros)
            scP_Hdu.data[bp]=np.nan
            sc = SpectralCube.read(scP_Hdu)
            del scP
            del scP_Hdu
            del zeros
            del bp
            sc.allow_huge_operations=True
            print('fov crop end')

            print('convolve start')

            try:
                sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
            except:
                cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
                cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
                if(cdelt_x>cdelt_y):
                    majorBase=cdelt_x
                    minorBase=cdelt_y
                elif(cdelt_x<cdelt_y):
                    majorBase=cdelt_y
                    minorBase=cdelt_x
                elif(cdelt_x==cdelt_y):
                    majorBase=cdelt_x
                    minorBase=cdelt_x
                BaseBeam = radio_beam.Beam(major=majorBase, minor=minorBase, pa=0*u.deg)

                sc = sc.with_beam(BaseBeam)

                beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)

                sc.allow_huge_operations=True
                print(sc.shape,np.nanmax(sc.hdu.data),'EA')
                #Requires me to edit convolve.py and set allow_huge =True
                sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
            print('convolve end')
            #
            #prepare a header for the reprojection
            #
            if(gal=="GC"):
                reheader = copy.deepcopy(sc.hdu.header)
            else:
                reheader = sc.hdu.header


            if(gal=="NGC253"):
                ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 
                if sc.header['cdelt1']>0:
                    pix_x    = (res/ovs).to(u.degree).value
                    origin_x = sc.longitude_extrema[0].to(u.degree).value

                else:
                    pix_x    = -1.*(res/ovs).to(u.degree).value
                    origin_x = (sc.longitude_extrema[1]).to(u.degree).value

                if sc.header['cdelt2']>0:
                    pix_y    = (res/ovs).to(u.degree).value
                    origin_y = sc.latitude_extrema[0].to(u.degree).value
                else:
                    pix_y    = -1.*(res/ovs).to(u.degree).value
                    origin_y = sc.latitude_extrema[1].to(u.degree).value

                npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
                npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                print(npix_x,npix_y,pix_x,origin_x,origin_y)
            elif gal=='GC':
                ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

                if sc.header['cdelt1']>0:

                    pix_x    = (res/ovs).to(u.degree).value
                    origin_x = sc.longitude_extrema[1].to(u.degree).value 
                    if(Force_Origins):

                        origin_x = Force_Value_x[0]#358.6

                else:

                    pix_x    = -(res/ovs).to(u.degree).value
                    origin_x = (sc.longitude_extrema[0]).to(u.degree).value 

                    if(Force_Origins):
                        origin_x = Force_Value_x[1]#.9

                if sc.header['cdelt2']>0:

                    pix_y    = (res/ovs).to(u.degree).value
                    origin_y = sc.latitude_extrema[0].to(u.degree).value

                    if(Force_Origins):

                        origin_y = Force_Value_y[0]#-.6

                else:

                    pix_y    = -(res/ovs).to(u.degree).value
                    origin_y = sc.latitude_extrema[1].to(u.degree).value

                    if(Force_Origins):

                        origin_y = Force_Value_y[1]#.6

                #manually put in the size to correct for the 360->0 difference and just because it doesnt seem to work
                print("LA",((sc.longitude_extrema[0])),(sc.longitude_extrema[1]-360*u.degree),np.abs(pix_x))
                print("MA",sc.latitude_extrema,np.abs(pix_y))
                npix_x   =int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
                npix_y   =int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
                if(Force_Origins):
                    npix_x   =int(np.diff(Force_Value_x)/np.abs(pix_x))
                    npix_y   =int(np.diff(Force_Value_y)/np.abs(pix_y))

                print(npix_x,npix_y)


            #Correct the header to the expected pixels for the new res

            reheader['cdelt1'] = pix_x
            reheader['cdelt2'] = pix_y

            reheader['naxis1'] = npix_x
            reheader['naxis2'] = npix_y

            reheader['crval1'] = origin_x
            reheader['crval2'] = origin_y

            reheader['crpix1'] = 0
            reheader['crpix2'] = 0
            if(gal=="GC"):
                reheader['CTYPE1'] = "GLON-SIN"
                reheader['CTYPE2'] = "GLAT-SIN"
            elif(gal=="NGC253"):
                pass
            try:
                del reheader['lonpole']
                del reheader['latpole']
                del reheader['wcsaxes']#Dont need these anymore, maybe?
                if gal =="GC":

                    del reheader['LBOUND1']
                    del reheader['LBOUND2']
                    del reheader['LBOUND3']
                    del reheader.cards['LBOUND1']
                    del reheader.cards['LBOUND2']
                    del reheader.cards['LBOUND3']

                    reheader['LBOUND1']=0
                    reheader['LBOUND2']=0
                    reheader['LBOUND3']=0

                print("12312412")
            except Exception as e:
                print(e)


                print("-"*60)
                traceback.print_exc(file=sys.stdout)

            # regrid cube to target pixel size

            print('repo start')

            print(np.nanmax(sc),np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)

            sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) #Have to change reproject so it deletes output.np before making a new one

            del sc

            new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
            new.allow_huge_operations=True
            new = new*sc2[0][0][0].unit
            #do this because scs dont like being modified
            del sc2
            sc2 = new
            del new
            print('repo end')
            print(np.nanmax(sc2),np.shape(sc2))#These should be a non zero float and the shape of the cube (30,~1000,~1000)


            Repo_Cube = sc2
            del sc2
            Repo_Cube.allow_huge_operations=True

            #do this again to crop the extra pixels off

            sc=Repo_Cube




            #Write the intermediary cubes that will then be spliced together
            #or not
            del Repo_Cube

            sc.write(str(str(i)+"_"+Cube_Name_Save ),overwrite=True)

        except Exception as e:
            print(e)
            print(kl)
            print("Failed")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)
def Splice_vels(Cube_Name_Load):
    
    for i in range(20):
        try:
            Cube_Name_Load_p = str(i)+"_"+Cube_Name_Load
            Cube_Name_Save = Cube_Name_Load

            sc=SpectralCube.read((Cube_Name_Load_p)) 
            print(np.shape(sc),Cube_Name_Load_p)
            print(sc.mask,type(sc.mask))

            if i==0:
                reheader = sc.header
                rewcs = sc.wcs


            print(type(sc))
            if i ==0:
                scW=SpectralCube.read((Cube_Name_Load_p))      
                mask = scW.mask.include() #Need to create a mask because it doesn't get splcied
            else:


                if i ==1:
                    scW = np.concatenate((scW[:].hdu.data,sc[:].hdu.data),dtype = type(sc))
                    print(type(scW))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
                else:
                    scW = np.concatenate((scW[:],sc.hdu.data[:]),dtype = type(sc))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))


            print(np.shape(scW))
        except Exception as e:
            print(e)
    def duh(lol):
        gp = np.where(lol!=np.nan)
        lol[gp]=True
        return lol #Anywhere that has data will be unmasked
    reheader["NAXIS3"] = len(scW)
    Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)
    scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)#The spliced cube

    scWsc.allow_huge_operations=True
    scWsc = scWsc*sc[0][0][0].unit#Add unit back in
    del sc


    scWsc.write(Cube_Name_Save,overwrite=True)
    #scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32.fits",overwrite=True)


    print("done")   

    
    
def Repo_Velocity(scp,Cube_Name_Save,spec_grid,G_width):

    scWsc_copy = scp
    fwhm_factor = np.sqrt(8*np.log(2))
    scWsc_copy.write("test0.fits",overwrite=True)

    scWsc_copy = scWsc_copy.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling
    print("Smoothed to Gaussian Kernel",G_width/fwhm_factor)
    if(Smoothe_2_5):
        scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)
    scWsc_copy.write("test1.fits",overwrite=True)

    scWsc_copy = scWsc_copy.spectral_interpolate(spectral_grid=spec_grid) # Match velocities to -250 251 range  

    scWsc_copy.write(Cube_Name_Save,overwrite=True)
    gc.collect()
    del scWsc_copy

    gc.collect()######################################################################



      
def Crop_nans(Cube_Name_Load,Prime_Beam,vel_prime,):
    

    Prime_Beam = Min_res*((kl*iter_factor+1))#
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        FOVp=[400,800]

        if(Match_to_HCO):
            Prime_Beam = 4.3*u.pc#Nico
            FOVp=[70,360]

    else:
        FOVp=FOV

    print(Prime_Beam)

    for km in range(5,6):
        vel_prime = min_vel*((km*iter_factor+1))
        if(km==5):
            vel_prime=2.5
            ovs=5
        if(Match_to_HCO):
            vel_prime=3.3
            ovs=3

        
        Cube_Name_Load = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'

        Cube_Name_Save=str("Cropped_"+Cube_Name_Load)
        sc = SpectralCube.read(Cube_Name_Load)
        datn = sc.hdu.data

        #Get right shape
        sx,sy,ex,ey=0,0,0,0
        for lmi in range(np.shape(datn[0,:,:])[0]):

            if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
                break
            for lmj in range(np.shape(datn[0,:,:])[1]):

                if(sx==0):            
                    if(np.nanmean(datn[0,lmi,:])>0 or np.nanmean(datn[0,lmi,:])<0):
                        sx=lmi


                if(sy==0):
                    if(np.nanmean(datn[0,:,lmj])>0 or np.nanmean(datn[0,:,lmj])<0):
                        sy=lmj

                if(ex==0):
                    if(np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])<0):
                        ex=np.shape(datn[0,:,:])[0]-lmi-1

                if(ey==0):
                    if(np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])>0 or np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])<0):
                        ey=np.shape(datn[0,:,:])[1]-lmj-1

                if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                    break
        print(sx,ex,sy,ey)


        sc = sc[:,sx:ex,sy:ey]

        sc_Hdu=sc.hdu
        zeros=((sc_Hdu.data[:,:,:]==0))
        bp = np.where(zeros)
        sc_Hdu.data[bp]=np.nan
        sc = SpectralCube.read(sc_Hdu)

        #Crop vel axis if needed

        sp=0
        for lmi in range(len(sc)):
            #Check to see if the slice has been repeated by the interpolation function
            if(np.round(np.nanmean(sc[lmi].hdu.data),5)==np.round(np.nanmean(sc[lmi+1].hdu.data),5)):

                sp = lmi+1
            else:
                break
        l = len(sc)-1
        ep=l
        for lmi in range(l):

            if(np.round(np.nanmean(sc[l-lmi].hdu.data),5)==np.round(np.nanmean(sc[l-lmi-1].hdu.data),5)):
                ep = l-lmi-1

            else:
                break


        sc.allow_huge_operations=True
        sc = sc[sp:ep]


        sc.write(Cube_Name_Save,overwrite=True)
        del sc
        del sc_Hdu
        gc.collect()
        print(kl,km,'ready')
        
        


        
        
        
def Noise_matching(Input_Cube_Match,Input_Cube_Noisy,Cube_Name_Save,Force_region=False,FVX=0,FVY=0,Force_Noise=False,FNV1=0,FNV2=0):
    
    
    

    #Find noise for ngc253
    

    Qp = Input_Cube_Noisy.with_spectral_unit(u.km/u.s,velocity_convention="radio") 
    Qp.allow_huge_operations=True

    Qp = Qp.to(u.K)#Jy to Kelvin

    datn = Qp.hdu.data
    del Qp


    Non_nan=((datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]>0)  | (datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]<0 ))

    NGCCO32_Noise = (np.nanstd(datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)],where= Non_nan)) #Noise K
    print(np.shape(datn))
    del datn





    Qp = Input_Cube_Match
    #match noise
    Qp.allow_huge_operations=True
    Q = Qp.to(u.K)#Jy to Kelvin
    del Qp
    datn=Q.hdu.data
    
    if gal =="GC":
        Non_nan=((datn[0,:,0:int(np.shape(datn)[2]/2)]>0)  | (datn[0,:,0:int(np.shape(datn)[2]/2)]<0 ))

        m = (np.nanstd(datn[0,:,0:int(np.shape(datn)[2]/2)],where= Non_nan)) #Noise K
    else:
        Non_nan=((datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]>0)  | (datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)]<0 ))

        m = (np.nanstd(datn[0,:,int(np.shape(datn)[2]/1.5):int(np.shape(datn)[2]-1)],where= Non_nan)) #Noise K

    if (Force_Noise):
        m = FNV1 #Force a noise Values .037? .115?
        NGCCO32_Noise=FNV2
    print(m,"Noise (K) matched to ",NGCCO32_Noise)





    npixels = np.product(Q.hdu.data.shape)

    target_noise = float(NGCCO32_Noise)
    actual_noise = m
    additional_sigma = np.sqrt(np.abs(target_noise**2 - actual_noise**2))

    additional_noise = np.random.normal(0., additional_sigma, npixels)
    additional_noise = np.reshape(additional_noise, Q.hdu.data.shape)


    fwhm_factor = np.sqrt(8*np.log(2))
    add_noise = np.zeros(np.shape(datn))
    for lmi in range(len(datn)):
        new_seed = np.random.randint(1e9)
        additional_noise = gauss_correlated_noise_2D(shape=(Q.hdu.data[6].shape[0],Q.hdu.data[6].shape[1]), sigma=additional_sigma, beam_gauss_width=5/fwhm_factor,randomseed=new_seed)
        pp=np.where(additional_noise!=np.nan)
        add_noise[lmi][pp]=additional_noise[pp]

    new_data = datn+add_noise
    QCopy = Q.hdu
    QCopy.data = new_data
    Q = SpectralCube.read(QCopy)
    del QCopy

    

    Q.write(Cube_Name_Save,overwrite=True)
    del Q
    print("Wrote to",Cube_Name_Save)
    
    
    
    
'''





#!py -m pip uninstall astropy
#!py -m pip install git+https://github.com/astropy/astropy
#!pip install emcee
!pip install corner
    



!py -m pip install git+https://github.com/radio-astro-tools/spectral-cube.git
!py -m pip install reproject
!py -m pip install git+https://github.com/radio-astro-tools/spectral-cube.git 
!py -m pip install pyspeckit
!py -m pip install regions
!py -m pip install astrodendro
!py -m pip  install wcsaxes 
!py -m pip  install ipympl
!py -m pip install dask
!py -m pip install radio_beam
!py -m pip install casa_formats_io
#try:
#    !pip install casa_formats_io --no-binary :all:
#except:
#    !pip install casa_formats_io --no-cache --no-binary :all:

!py -m pip  install spectral_cube 
!py -m pip  install typing 
!py -m pip install mypy
!py -m pip  install typing_extensions 



'''



if(os.path.exists("./Result Files")):
    print("Results will be saved to ./Result Files")
else:
    %mkdir "./Result Files"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16
/home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
/home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
1.23.1 Numpy
Results will be saved to ./Result Files
